[<img src="plexo-logo.png" alt="Drawing" style="width: 250px;"/>](https://plexocap.com/)

# [Plexo Capital Overview](https://plexocap.com/)

## Know Your LP 

[Plexo Capital](https://plexocap.com/) is an institutional investor allocating capital globally to the startup ecosystem. On the LP side, we targeted sub USD 100M funds in fund I where their primary entry point for investment is pre seed or seed and they are leading the majority of their deals. We are still planning the strategy for fund II, and fund II will continue the focus on sub USD 100M funds and likely include some funds over USD 100M with an entry point of seed and small Series A rounds.

Being data driven, we are disciples of the [power law distribution of returns](http://reactionwheel.net/2015/06/power-laws-in-venture.html) in early stage venture capital. As a result, we focus on GPs that can lead the majority of their deals which helps with both ownership levels and has the byproduct of minimizing the overlap in the underlying portfolios of our GPs. That allows us to cast as wide a net as possible to capture outliers.

We source our direct investments from the portfolios of our GPs. Since an important component of our strategy is to provide downstream deal flow to our LPs, which include [Alphabet/GV](https://www.gv.com/), [Intel Capital](https://www.intel.com/content/www/us/en/intel-capital/overview.html), [Cisco Investments](https://www.ciscoinvestments.com/) and [RBC](https://www.rbccm.com/en/about-us.page), we try to align our selection with those GPs we feel will construct a portfolio of companies that will be of interest to those LPs that also invest directly.  As far as sector focus, we prefer generalist funds that invest mainly in enterprise, marketplaces + technology enabled consumer. If we lean in any direction, it would be towards enterprise.

Based on our data and research, fund I targeted ~21 LP commitments. At this stage, we have committed to 20 funds (including the original five LP commitments that we made at GV which came over to the Plexo Capital portfolio) in Plexo Capital I. If we make another LP commitment in fund I (highly unlikely), we are looking at generalist funds in geographies outside of the United States and funds in the US with a thesis that might benefit in a post COVID-19 environment. 

Many of our commitments were made to funds where we built relationships over the first two years while we incubated inside GV.  We also like to spend about 10 hours getting to know a manager in a combination of 1:1 meetings, phone calls and catch ups at conferences. Therefore, if we are off from a timing perspective, we want to be prepared in the future for our subsequent funds. 

It would be great to schedule a meeting to learn more about your fund. To maintain a level playing field for access, we take a page our of the playbook of our LP [Kapor Capital](https://www.kaporcapital.com/) and ask all GPs + entrepreneurs to fill out the [form below](https://airtable.com/shru2gLH98tLjxIp8) so that we can gather more information about your fund and prepare for a productive meeting.  After the form submission, it will redirect to a page to schedule a meeting with [Vishal](https://www.linkedin.com/in/vtripathi30/) + [Sam](https://www.linkedin.com/in/sam-toney/).

https://airtable.com/shru2gLH98tLjxIp8

---

## Portfolio Construction

This webpage is built to help our prospective GPs have a run at portfolio construction through a simulated experience. Enter details about your investments and see the probabilistic performance of your fund. The data for this analysis is sourced from [Pitchbook](https://my.pitchbook.com/?pcc=366074-38) and the [Angel Investor Performance Project](http://reactionwheel.net/2013/03/aipp-data-summary-angel-investing.html) by [Kauffman Foundation](https://www.kauffman.org/).

More factors will be added continuously on this page. For any suggestions or improvements please write to <vishal@plexocap.com>. 

---

## Industry Analysis


In [1]:
#imports
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import scipy
import powerlaw
import bqplot
from bqplot import pyplot as blt
import ipywidgets as widgets
import warnings
plt.style.use('seaborn-darkgrid')
warnings.filterwarnings("ignore")

Below see a discrete analysis for the different sectors and their returns distribution (legacy values).

Select industry verticals for histogram of returns distribution.


In [2]:
#Data Research

### Alibek's data for discrete analysis

disc = pd.read_csv('discreteDist.csv', index_col = 0).T
mult = pd.read_csv('multiples.csv', index_col = 0).T

sectors = list(mult.columns)


In [3]:
def plot_func(secs):
    df = disc[list(secs)]
    df.plot(kind = 'bar')
    plt.xlabel("Return Multiples")
    plt.ylabel("Percentage of Companies")
    plt.show()
    

widg1 = widgets.interactive(plot_func, secs=widgets.SelectMultiple(
    options=sectors,
    value=[sectors[3],sectors[-1]],
    description='Verticals',
    disabled=False
    ))

display(widg1)

interactive(children=(SelectMultiple(description='Verticals', index=(3, 17), options=('Ad/Marketing tech', 'AI…

In [4]:
#Exact Multiples (Capped at 50x) :

In [5]:
def mutiples(secs):
    display(mult[list(secs)])
    
widg2 = widgets.interactive(mutiples, secs=widgets.SelectMultiple(
    options=sectors,
    value=[sectors[3],sectors[-1]],
    description='Verticals',
    disabled=False
    ))

#display(widg2)

In [6]:
#grid = widgets.GridspecLayout(1,2, height='400px')
#grid[:,0] = widg1
#grid[:,1] = widg2
#display(grid)

In [7]:
#Moments for discrete distribution :

In [8]:
mu = np.sum(np.multiply(mult,disc))
mom2 = np.sum(np.multiply(mult**2,disc))
var = mom2-mu**2
stdDev = np.sqrt(var)

moments = pd.DataFrame([mu,var,stdDev]).T
moments = moments.rename_axis('Sectors')
moments = moments.rename(columns= {0:'Mean',1:'Variance',2:'Standard Error'})

#display(moments)

Unlike the return multiples in the discrete setting, we could also have a continuous analysis of power law distribution. You can read more about the distribution here - [Pareto Distribution](https://en.wikipedia.org/wiki/Pareto_distribution)

We would define the probability distribution of return multiples in the following way :

$$ \begin{equation}
    p(x) = \begin{cases}
    \frac{\alpha-1}{\alpha}, & \text{for $x<1$}\\
    \frac{(\alpha - 1)}{\alpha} x^{-\alpha}, & \text{for $x>=1$}
    \end{cases} \end{equation} $$
    
Based on empirical evidence, we have found $\alpha$ to be between 1.9-2.5. We will calibrate this $\alpha$ for each vertical based on data we have. For the analysis below and for stability we are choosing TVPI $\alpha = 2.42$. 

This number comes from the angellist data and research paper (page 8) - [Source](https://angel.co/pdf/growth.pdf), based on the log likelihood maximization method derived originally from the works of [Clauset et al (2009)](https://arxiv.org/pdf/0706.1062.pdf).

One assumption is that the probability of firms between 0 and 1x returns are considered constant but given the alpha this is a conservative approach for this analysis.

The return multiple probability distribution function looks like this -

In [10]:
def pdf(x, alpha = 2.42):
    if x<=1:
        return (alpha-1)/(alpha)
    res = (alpha-1)*np.power(x,-alpha)/alpha
    return res

alpha = 2.42
x = np.linspace(0,10,1000)
y = [pdf(i,alpha) for i in x]
blt.plot(x,y)
blt.title('Continuos probability distribution for return multiples')
blt.xlabel('Return Multiples')
blt.ylabel('Probabaility')
blt.show()

### Mean is calculated based on normal distribution behaviour
directmean = (alpha-1)/(alpha*(alpha-2))+(alpha-1)/(2*alpha)

### Downside risk is calculated for firms going below 1x. (So only constant part of the equation)
directDR = (alpha-1)/(3*alpha)

## Optimization for optimal weights in each bucket

The aim of this exercise is to get optimal weights for each of the buckets decribed above. The method used to achieve this is a convex optimization problem similar to the [mean-variance optimization](https://www.investopedia.com/terms/m/meanvariance-analysis.asp).

The aim would be to maximize the returns coming from each bucket and at the same time minimizing the downside risk. Given the analysis is being done for private markets and the returns are not normally distributed, the use of downside risk is the perfect factor to not penalise the underliers for high return. The measurement we use for this is the semi-variance (described [here](https://www.investopedia.com/terms/s/semivariance.asp)). This idea is inspired by the use of [Sortino Ratio](https://en.wikipedia.org/wiki/Sortino_ratio) and discussions by [Fama-French](https://famafrench.dimensional.com/questions-answers/qa-semi-variance-a-better-risk-measure.aspx).

For each of the buckets described above the mean is calculated the usual way while the downside risk is calculated by the formulas described in the above links. For our Core GPs and for Direct Investments, the risk is considered only below a 1x return, while for the Beta GPs and Senior GPs, risk is considered below the mean returns as suggested by the papers. This is because of our fund structure preference with the GPs in the 15MM - 100MM range and a better deal flow compensating for the risk.

This is how the returns and the downside-risk (semi-variance) for each bucket looks like - 

In [13]:
riskRet = pd.DataFrame([(betaGPmean,betaGPDR),(coreGPmean,coreGPDR),(senGPmean,senGPDR),(directmean,directDR)],\
                       columns = ['Mean Return','Semi-Variance'], index = ['Beta GPs','Core GPs','Senior GPs','Direct Investments'])
display(riskRet)

,Mean Return,Semi-Variance
Beta GPs,1.445960,0.239871
Core GPs,1.657934,0.127825
Senior GPs,1.490009,0.241929
Direct Investments,1.690476,0.195592


Now we will use convex optimization to maximise for returns while minimizing the down-side risk. The problem is defined as -

$$ \begin{equation*}
\begin{aligned}
& \underset{w}{\text{minimize}}
& & -\frac{w\cdot r}{w\cdot \sigma \cdot w^T} \\
& \text{subject to}
& & \sum w = 1, \; 0 \leq w \leq 1.
\end{aligned}
\end{equation*}  $$

where, $w$ and $r$ are vectors and $\sigma$ is a matrix with the downside risk on the diagonal and 0 on the rest places.

After running the analysis we get the following results -

In [14]:
def fitness(W):
    mu = np.sum(np.array(riskRet['Mean Return'])*np.array(W))
    var = np.sum(np.square(np.array(W))*np.array(riskRet['Semi-Variance']))
    util = mu / np.sqrt(var)      
    return -util

n = 4
W = np.ones([n])/n                  # start with equal weights
b_ = [(0.,1.) for i in range(n)]    # weights between 0%..100%. 
c_ = ({'type':'eq', 'fun': lambda W: sum(W)-1. })   # Sum of weights = 100%

optimized = scipy.optimize.minimize(fitness, W, constraints=c_, bounds=b_)

if not optimized.success: 
    raise BaseException(optimized.message)
    
weights = optimized.x  # Return optimized weights

weightsDF = pd.DataFrame(weights*100,columns = ['Weights in Percentage'],index = ['Beta GPs','Core GPs','Senior GPs','Direct Investments'])

display(weightsDF)

,Weights in Percentage
Beta GPs,17.833307
Core GPs,38.373775
Senior GPs,18.220576
Direct Investments,25.572341
